# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
import json

## Dataset

Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'heartDiseaseHyperDrive'
experiment=Experiment(ws, experiment_name)

In [3]:
#Compute CLuster
compute_cluster_name = 'project'
try:
    compute_cluster = ComputeTarget(workspace=ws,name=compute_cluster_name)
    print("Found existing compute cluster")
except ComputeTargetException:
    print("Creating new Compute Cluster")
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_D2_V2', max_nodes = 4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_cluster_config)
    compute_cluster.wait_for_completion(show_output = True)
print(compute_cluster.get_status().serialize())

Found existing compute cluster
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-10T10:54:51.204000+00:00', 'errors': None, 'creationTime': '2021-01-10T10:50:19.835696+00:00', 'modifiedTime': '2021-01-10T10:50:35.780465+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Hyperdrive Configuration

Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval = 2 , slack_factor = 0.1)

#Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--max_iter' : choice([100,200,300,400]),
    '--C' : choice([0.001, 0.01, 0.1, 1, 10, 100, 1000])
})

# Create your estimator and hyperdrive config
estimator = SKLearn(source_directory = './', entry_script = 'train.py', compute_target = compute_cluster)

hyperdrive_run_config = HyperDriveConfig(
    estimator = estimator,
    hyperparameter_sampling = param_sampling,
    primary_metric_name = 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    policy = early_termination_policy,
    max_total_runs = 20,
    max_concurrent_runs = 4
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
# Submit your experiment
hyperdrive_run = experiment.submit(config = hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

 In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(hyperdrive_best_run.get_details()['runDefinition']['arguments'])
hyperdrive_best_run.get_file_names()

['--C', '100', '--max_iter', '400']


['azureml-logs/55_azureml-execution-tvmps_c92231fbec862850c5704cc8e8183627f7cd1abd155a576554ddddf2911c7893_d.txt',
 'azureml-logs/65_job_prep-tvmps_c92231fbec862850c5704cc8e8183627f7cd1abd155a576554ddddf2911c7893_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_c92231fbec862850c5704cc8e8183627f7cd1abd155a576554ddddf2911c7893_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/100_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/LogisticRegression.pkl']

In [8]:
hyperdrive_best_run_metrics = hyperdrive_best_run.get_metrics()
for metric_name in hyperdrive_best_run_metrics:
    metric = hyperdrive_best_run_metrics[metric_name]
    print(metric_name, metric)

Regularization Strength: 100.0
Max iterations: 400
Accuracy 0.8888888888888888


In [9]:
# Save the best model
model=hyperdrive_best_run.register_model(model_name='hyperdrivebestrun', model_path='outputs/LogisticRegression.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

 In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
myenv=Environment.from_conda_specification(name='myenv',file_path='conda_env.yml')
inference_config = InferenceConfig(entry_script='score.py',environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True)
service= Model.deploy(workspace=ws,
                     name='heart-disease',
                     models=[model],
                     inference_config=inference_config,
                     deployment_config=aci_config,
                     overwrite=True)

In [22]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [30]:
#Standout Suggestions - Application Insights
service.update(enable_app_insights=True)

 In the cell below, send a request to the web service you deployed to test it.

In [24]:
ds = TabularDatasetFactory.from_delimited_files("https://raw.githubusercontent.com/GowthamiWudaru/heart-Disease-Prediction-With-Azure/main/heartDisease.csv")
df=ds.to_pandas_dataframe().drop(['num'], axis=1)

In [31]:
input_payload = json.dumps({
    'data': df.sample(3).to_dict()
})
output = service.run(input_payload)
print(output)

{"result": [0, 1, 1]}


 In the cell below, print the logs of the web service and delete the service

In [32]:
service.delete()

In [33]:
try:
    compute_cluster.delete()
    print("Deleted compute cluster")
except ComputeTargetException:
    print("Compute Cluster Not found")

Deleted compute cluster
